# Добро пожаловать в практическую часть


Вы находитесь в онлайн-сервисе Google Colab. Этот сервис используется для написания кода и предоставляет мощности для обучения моделей.

В рамках практической части мы пройдемся по основным этапам обучения модели. Весь код уже написан и логически разбит по ячейкам. Чтобы все отработало неоходимо последовательно запускать ячейки. Для запуска ячейки необходимо нажать круглую кнопку с треугольником в левом верхнем углу запускаемой ячейки.


---

### Нулевой шаг

Перед началом работы переключимся на использование мощностей видеокарты. Это позволит обучать модель значительно быстрее.

Для этого нужно нажать вверху "Изменить", затем "Настройки блокнота" и убедиться, что под словами "Аппартаный ускоритель" написано "GPU"

### Установка модулей

In [1]:
# Первым делом нам нужно установить дополнительные модули для Python.
# pyTelegramBotAPI необходим для работы с телеграмботом
!pip install pyTelegramBotAPI
# transformers необходим для работы с моделями
!pip install transformers

# Для запуска ячейки необходимо нажать круглую кнопку с треугольником в левом верхнем углу запускаемой ячейки

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 157 kB 7.9 MB/s 
  Created wheel for pyTelegramBotAPI: filename=pyTelegramBotAPI-4.5.1-py3-none-any.whl size=138116 sha256=fa6483dad68ade959a7479350b779afc632f9e2c16442b01f6db729c7846f44f
  Stored in directory: /root/.cache/pip/wheels/9a/90/69/89e4e68c819633d933753827fd4f3a22987e162b7404b092b5
Successfully built pyTelegramBotAPI
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.4 MB 6.0 MB/s 
     |████████████████████████████████| 6.6 MB 13.6 MB/s 
     |████████████████████████████████| 596 kB 50.5 MB/s 
     |████████████████████████████████| 86 kB 3.0 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


## Активация необходимых модулей и доступа к гугл-диску

In [2]:
# Какие-то модули уже стоят в питоне изначально, какие-то мы поставили выше. Сейчас их необходимо автивировать

import pandas as pd
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import GPT2Tokenizer, GPT2LMHeadModel, AdamW, get_linear_schedule_with_warmup,AutoTokenizer,AutoModelForCausalLM
from tqdm import tqdm, trange
import torch.nn.functional as F
import re

In [3]:
# Разрешаем нашему коду смотреть в гугл-диск
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Исходные данные

In [4]:
# Открываем таблицу с исходными данными с помощью модуля pandas
data = pd.read_csv('/content/drive/MyDrive/Практическое_занятие/данные/reklama.csv', sep='\t')
data.head(10)

index                                               text  \
0      0  Заказывайте в новый пункт выдачи Ozon.ru  ул. ...   
1      1  Воспользуйтесь банкоматом ПСБ – пр-т Вернадско...   
2      2  ОСАГО с бесплатной доставкой по г.Соколу!\nhtt...   
3      3  Воспользуйтесь банкоматом ПСБ – пр-т Ленинград...   
4      5  Воспользуйтесь банкоматом ПСБ – пр-т Маршала Ж...   
5      6  Ищете новый Renault? Купите выгодно в Рольф! +...   
6      8  Воспользуйтесь банкоматом ПСБ – пр-т Мира, д. ...   
7     11  Воспользуйтесь банкоматом ПСБ – пр-т Мичуринск...   
8     12  Воспользуйтесь банкоматом ПСБ – пр-т Новоясене...   
9     13  Воспользуйтесь банкоматом ПСБ – Рязанский пр-т...   

                                          clean_text  
0  Заказывайте в новый пункт выдачи Ozon.ru  ул. ...  
1  Воспользуйтесь банкоматом ПСБ – пр-т Вернадско...  
2  ОСАГО с бесплатной доставкой по г.Соколу!\n <А...  
3  Воспользуйтесь банкоматом ПСБ – пр-т Ленинград...  
4  Воспользуйтесь банкоматом ПСБ – пр-т Маршала Ж...  
5  Ищете новый Renault? Купите выгодно в Рольф! <...  
6  Воспользуйтесь банкоматом ПСБ – пр-т Мира, д. ...  
7  Воспользуйтесь банкоматом ПСБ – пр-т Мичуринск...  
8  Воспользуйтесь банкоматом ПСБ – пр-т Новоясене...  
9  Воспользуйтесь банкоматом ПСБ – Рязанский пр-т...

In [5]:
# Добавим к каждому тексту тег окончания текста, чтобы модель понимала где обычно они заканчиваются
texts = '<|endoftext|>'.join([text for text in data['clean_text']])

In [6]:
# Проверим, как выглядит текст после изменений. Возьмем первые 1000 символов
texts[:1000]

'Заказывайте в новый пункт выдачи Ozon.ru  ул. Немировича-Данченко д. 2<|endoftext|>Воспользуйтесь банкоматом ПСБ – пр-т Вернадского, д. 37, корп. 1 Б\n\n\n\n<|endoftext|>ОСАГО с бесплатной доставкой по г.Соколу!\n <АДРЕС_САЙТ><|endoftext|>Воспользуйтесь банкоматом ПСБ – пр-т Ленинградский, д. 76 А\n\n\n\n\n<|endoftext|>Воспользуйтесь банкоматом ПСБ – пр-т Маршала Жукова, д. 38, стр. 1\n\n\n\n<|endoftext|>Ищете новый Renault? Купите выгодно в Рольф! <ТЕЛЕФОННЫЙ_НОМЕР> Звоните!<|endoftext|>Воспользуйтесь банкоматом ПСБ – пр-т Мира, д. 116 А\n\n\n\n<|endoftext|>Воспользуйтесь банкоматом ПСБ – пр-т Мичуринский, д. 27\n\n\n\n<|endoftext|>Воспользуйтесь банкоматом ПСБ – пр-т Новоясеневский, д. 7\n\n\n\n\n<|endoftext|>Воспользуйтесь банкоматом ПСБ – Рязанский пр-т, д. 16, стр. 4\n\n\n\n\n\n<|endoftext|>с 18 по 22 апреля 2022г. Эксклюзивные дома из клееного бруса, в комплектации: фундамент, стены, крыша, окна, покраска. Регистрируйтесь:  <АДРЕС_САЙТ><|endoftext|>До <ДАТА> бесплатная доставка 

In [7]:
# Сохраним наши новые данные в файл
with open('/content/drive/MyDrive/Практическое_занятие/данные/r_text.txt','w', encoding='utf-8') as f:
  f.write(texts)

In [8]:
# Посмотрим, что они корректно сохранились
with open('/content/drive/MyDrive/Практическое_занятие/данные/r_text.txt','r', encoding='utf-8') as f:
  opened_texts = f.read()

opened_texts[:1000]

'Заказывайте в новый пункт выдачи Ozon.ru  ул. Немировича-Данченко д. 2<|endoftext|>Воспользуйтесь банкоматом ПСБ – пр-т Вернадского, д. 37, корп. 1 Б\n\n\n\n<|endoftext|>ОСАГО с бесплатной доставкой по г.Соколу!\n <АДРЕС_САЙТ><|endoftext|>Воспользуйтесь банкоматом ПСБ – пр-т Ленинградский, д. 76 А\n\n\n\n\n<|endoftext|>Воспользуйтесь банкоматом ПСБ – пр-т Маршала Жукова, д. 38, стр. 1\n\n\n\n<|endoftext|>Ищете новый Renault? Купите выгодно в Рольф! <ТЕЛЕФОННЫЙ_НОМЕР> Звоните!<|endoftext|>Воспользуйтесь банкоматом ПСБ – пр-т Мира, д. 116 А\n\n\n\n<|endoftext|>Воспользуйтесь банкоматом ПСБ – пр-т Мичуринский, д. 27\n\n\n\n<|endoftext|>Воспользуйтесь банкоматом ПСБ – пр-т Новоясеневский, д. 7\n\n\n\n\n<|endoftext|>Воспользуйтесь банкоматом ПСБ – Рязанский пр-т, д. 16, стр. 4\n\n\n\n\n\n<|endoftext|>с 18 по 22 апреля 2022г. Эксклюзивные дома из клееного бруса, в комплектации: фундамент, стены, крыша, окна, покраска. Регистрируйтесь:  <АДРЕС_САЙТ><|endoftext|>До <ДАТА> бесплатная доставка 

### Запускаем обучение модели

In [9]:
%%time
!export PYTHONPATH=${PYTHONPATH}:/ru-gpts/
!CUDA_VISIBLE_DEVICES=0 python /content/drive/MyDrive/Практическое_занятие/ru-gpts/pretrain_transformers.py \
    --output_dir=our_finetune_model \
    --model_type=gpt2 \
    --model_name_or_path=sberbank-ai/rugpt3small_based_on_gpt2 \
    --do_train \
    --train_data_file=/content/drive/MyDrive/Практическое_занятие/данные/r_text.txt \
    --per_gpu_train_batch_size 1 \
    --gradient_accumulation_steps 1 \
    --num_train_epochs 2 \
    --block_size 1024 \
    --overwrite_output_dir

06/18/2022 09:08:13 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False, 16-bits training: False
Downloading: 100% 608/608 [00:00<00:00, 670kB/s]
Downloading: 100% 1.63M/1.63M [00:00<00:00, 4.46MB/s]
Downloading: 100% 1.21M/1.21M [00:00<00:00, 3.32MB/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:973: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,
Downloading: 100% 526M/526M [00:07<00:00, 73.9MB/s]
06/18/2022 09:08:41 - INFO - __main__ -   Training/evaluation parameters Namespace(adam_epsilon=1e-08, block_size=1024, cache_dir=None, config_name=None, device

## Генерируем текст

In [10]:
model_name_or_path = "/content/our_finetune_model" # Указываем путь до папки с нашей моделью
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path) # Активируем токенизатор - инуструмент, который превратит наш текст в нужный для модели формат
gpt = AutoModelForCausalLM.from_pretrained(model_name_or_path) # Активируем модель

### Запускаем генератор текста

Значение prompt отвечает за начало генерируемого текста. Его можно изменить, для лучшего результата сохранив тематику данных.

Параметры, которые можно позименять, чтобы изменить результат генерации:

*   max_length - максимальная длинна генерируемого текста в символах

*   num_return_sequences - сколько текстов сгенерирует модель. Чем больше текстов, тем дольше модель занимается генерацией

*   temperature - Чем выше этот параметр, тем более разнообразные тексты получаются. Однако, с повышением этого параметра повышается и вероятность, что текст получится менее похожим на человеческий

In [16]:
prompt = 'Заказные убийства' # Начальные слова, которые подаются модели. С них и будет начинаться генерируемый текст

input_ids = tokenizer.encode(prompt, return_tensors="pt") # С помощью токенизатора преобразуем начальные слова в формат для модели

# Функция генерации текста
# Параметры, которые можно позименять, чтобы изменить результат генерации:
#   max_length - максимальная длинна генерируемого текста в символах
#   num_return_sequences - сколько текстов сгенерирует модель. Чем больше текстов, тем дольше модель занимается генерацией
#   temperature - Чем выше этот параметр, тем более разнообразные тексты получаются. 
#                 Однако, с повышением этого параметра повышается и вероятность, что текст получится менее похожим на человеческий
generated_text_samples = gpt.generate(
    input_ids, 
    max_length=60,  
    num_return_sequences=5,
    no_repeat_ngram_size=3,
    repetition_penalty=1.5,
    top_p=1.,
    temperature=1,
    do_sample=True,
    top_k=125,
    early_stopping=True
)

# Выводим результат генерации
for i, beam in enumerate(generated_text_samples):
  print("{}: {}".format(i,tokenizer.decode(beam, skip_special_tokens=False).split('<|endoftext|>')[0]))
  print()

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


0: Заказные убийства с гарантией 5 лет. Подробнее -  <АДРЕС_САЙТ>

1: Заказные убийства и банальные ошибки в юриспруденции, тел. <ТЕЛЕФОННЫЙ_НОМЕР> 

2: Заказные убийства для мужчин. Заберите заказ в приложении   <АДРЕС_САЙТ>

3: Заказные убийства и заказ на "свежее мясо". <ДАТА> в Порядке: 2 заказа с доставкой  от 55 руб! Приходите
на ул. Мичурина, д 28Б

4: Заказные убийства в Уфе
Услуги сертифицированы.



### Создаем ТГ Бота

Откройте мессенджер Telegram, войдите в вашу учетную запись.


1.   Введите в поле поиска @BotFather и выберите бота.

2.   Нажмите «Запустить» для активации бота BotFather. В ответ вы получите список команд по управлению ботов.

3.   Выберите или напечатайте и отправьте команду /newbot.

4.   Дайте имя боту — пользователи увидят это имя при общении с ботом. И никнейм бота — по нему можно будет найти бота в Telegram. Никнейм должен быть уникальным, не повторять существующие в базе и заканчиваться на слово «bot».

После того как вы выберите подходящее имя бот будет создан. Вы получите сообщение со ссылкой на бота t.me/<никнейм_бота>

Для подключения модели к боту нам понадобится токен


In [ ]:
import telebot
import torch
from transformers import AutoTokenizer,AutoModelForCausalLM
from transformers import GPT2LMHeadModel, GPT2Tokenizer


def model(text, mo, max_length=42):
  input_ids = tokenizer.encode(text, return_tensors="pt")

  generated_text_samples = mo.generate(
      input_ids, 
      max_length=60,  
      num_return_sequences=1,
      no_repeat_ngram_size=3,
      repetition_penalty=2.5,
      top_p=1.,
      temperature=1,
      do_sample=True,
      top_k=125,
      early_stopping=True
  )
  return tokenizer.decode(generated_text_samples[0], skip_special_tokens=False).split('<|endoftext|>')[0]

bot = telebot.TeleBot('5431478716:AAHCEmuzoIgUL-Z2A7kQb7GeZkkjVUavFkY') # В этой строчке необходимо вставить токен от Вашего бота

@bot.message_handler(commands=["start"])
def start(m, res=False):
    bot.send_message(m.chat.id, 'Пожалуйста, напишите как Вы хотите начать свой рекламнй текст')

@bot.message_handler(content_types=["text"])
def handle_text(message):
    bot.send_message(message.chat.id, model(message.text, gpt))
    bot.send_message(message.chat.id, model(message.text, gpt))
    bot.send_message(message.chat.id, 'Если хотите продолжить, напишите, пожалуйста, начало следующего рекламного текста')

bot.polling(none_stop=True, interval=0)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generati